In [3]:
import cv2
import numpy as np
import dlib
from math import hypot
from playsound import playsound
import time
from threading import Timer, Event

cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

font = cv2.FONT_HERSHEY_SIMPLEX

total_frame = 0

frame_count = 0

count = 0

blink = 0

def every_so_often():
    if not done.is_set():
        print('Do the thing you want to every so often')
        Timer(5.0, every_so_often).start()

done = Event()
           
def get_blinking_ratio(eye_points , facial_landmarks):
        #x = facial_landmarks.part(eye_points[0]).x
        #y = facial_landmarks.part(eye_points[0]).y
        #cv2.circle(frame , (x , y) , 3 , (0 , 0 , 255) , 2)
        left_point = (facial_landmarks.part(eye_points[0]).x , facial_landmarks.part(eye_points[0]).y)
        right_point = (facial_landmarks.part(eye_points[3]).x , facial_landmarks.part(eye_points[3]).y)
        
        mid_point_x = int((facial_landmarks.part(eye_points[0]).x + facial_landmarks.part(eye_points[3]).x)/2)
        mid_point_y = int((facial_landmarks.part(eye_points[0]).y + facial_landmarks.part(eye_points[3]).y)/2)
        
        hor_line = cv2.line(frame , left_point , right_point , (0 , 255 , 0) , 1)
        
        top_point = (int((facial_landmarks.part(eye_points[1]).x + facial_landmarks.part(eye_points[2]).x)/2)  , int((facial_landmarks.part(eye_points[1]).y + facial_landmarks.part(eye_points[2]).y)/2))
        bottom_point = (int((facial_landmarks.part(eye_points[5]).x + facial_landmarks.part(eye_points[4]).x)/2)  , int((facial_landmarks.part(eye_points[5]).y + facial_landmarks.part(eye_points[4]).y)/2))
        
        ver_line = cv2.line(frame , top_point , bottom_point , (0 , 255 , 0) , 1)
        
        #x = int((facial_landmarks.part(eye_points[0]).x + facial_landmarks.part(eye_points[3]).x)/2)
        #y = int((((facial_landmarks.part(eye_points[1]).y + facial_landmarks.part(eye_points[2]).y)/2) + ((landmarks.part(eye_points[1]).y + landmarks.part(eye_points[2]).y)/2))/2)
        
        cv2.circle(frame , (mid_point_x , mid_point_y) , 1 , (255 , 0 , 0) , 1)
        
        hor_line_length = hypot((left_point[0] - right_point[0]) , (left_point[1] - right_point[1]))
        ver_line_length = hypot((top_point[0] - bottom_point[0]) , (top_point[1] - bottom_point[1]))
        ratio =  (hor_line_length/ver_line_length)
        
        if ver_line_length != 0:
            return ratio

    
while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame , cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
        #print (face)
        #x , y = face.left() , face.top()
        #x1 , y1 = face.right() , face.bottom()
        #cv2.rectangle(frame , (x , y) , (x1 , y1) , (0 , 255 , 0) , 2)
        
        landmarks = predictor(gray , face)
        
        left_eye_ratio = get_blinking_ratio([36 , 37 , 38 , 39 , 40 , 41] , landmarks)
        right_eye_ratio = get_blinking_ratio([42 , 43 , 44 , 45 , 46 , 47] , landmarks)
        
        total_frame = total_frame + 1
        
        frame_count = frame_count + 1
        
        if left_eye_ratio > 5.5 or right_eye_ratio > 5.5:
            cv2.putText(frame , "BLINKING" , (50 , 150) , font , 3 , (255 , 0 , 0))
            blink = blink + 1
            count = count + 1
            frec = frec + 1
            if count >= 42:
                cv2.putText(frame , "DROWZINESS DETECTED!!!!" , (50 , 150) , font , 10 , (0 , 255 , 0))
                count = 0
            if frame_count >= 3600:
                blink_frequency = frec / 120
                avg_blink_frequency = (blink * 30) / (total_frame)
                frame_count = 0
                
        else:
            count = 0
        #gaze detection
        left_eye_region = np.array([(landmarks.part(36).x , landmarks.part(36).y),
                                   (landmarks.part(37).x , landmarks.part(37).y),
                                   (landmarks.part(38).x , landmarks.part(38).y),
                                   (landmarks.part(39).x , landmarks.part(39).y),
                                   (landmarks.part(40).x , landmarks.part(40).y),
                                   (landmarks.part(41).x , landmarks.part(41).y)] , np.int32)
        #print(left_eye_region)
        #cv2.polylines(frame , [left_eye_region] , True , (0 , 0 , 255) , 1)
        
        height , width , _ = frame.shape
        mask = np.zeros((height , width) , np.uint8)
        
        cv2.polylines(mask , [left_eye_region] , True , 255 , 1)
        cv2.fillPoly(mask , [left_eye_region] , 255)
        
        left_eye = cv2.bitwise_and(gray , gray , mask = mask)
        
        min_x = np.min(left_eye_region[: , 0])
        max_x = np.max(left_eye_region[: , 0])
        min_y = np.min(left_eye_region[: , 1])
        max_y = np.max(left_eye_region[: , 1])
        
        gray_eye = left_eye[min_y : max_y , min_x : max_x]
        #gray_eye = cv2.cvtColor(eye , cv2.COLOR_BGR2GRAY)
        _, threshold_eye = cv2.threshold(gray_eye , 70 , 255 , cv2.THRESH_BINARY)
        height , width = threshold_eye.shape
        left_side_threshold = threshold_eye[0: height , 0: int(width/2)]
        left_side_white = cv2.countNonZero(left_side_threshold)
        
        right_side_threshold = threshold_eye[0: height , int(width/2) : width]
        right_side_white = cv2.countNonZero(right_side_threshold)
        
        if right_side_white !=0:
            gaze_ratio = left_side_white/right_side_white
            cv2.putText(frame , str(gaze_ratio) , (50 , 150) , font , 2 , (0 , 0 , 255) , 3)
        #cv2.putText(frame , str(left_side_white) , (50 , 100) , font , 2 , (0 , 0 , 255) , 3)
        #cv2.putText(frame , str(right_side_white) , (50 , 150) , font , 2 , (0 , 0 , 255) , 3)
        
        threshold_eye = cv2.resize(threshold_eye , None , fx = 5 , fy = 5)
        eye = cv2.resize(gray_eye , None , fx = 5 , fy = 5)
        
        #cv2.imshow("Eye" , eye)
        cv2.imshow("threshold_eye" , threshold_eye)
        cv2.imshow("left threshold" , left_side_threshold)
        cv2.imshow("right threshold" , right_side_threshold)
        #cv2.imshow("left_eye" , left_eye)


    cv2.imshow("Frame" , frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()
print(count)

0
